In [1]:
!nvidia-xmi

/bin/bash: line 1: nvidia-xmi: command not found


In [2]:
import os
import time

from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

In [3]:
model_id = "vectara/hallucination_evaluation_model"

In [9]:
tt = time.time()
model = AutoModelForSequenceClassification.from_pretrained(model_id, trust_remote_code=True)
print(f"\nTime taken to load model {model_id}: {int(time.time() - tt)} seconds")

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.



Time taken to load model vectara/hallucination_evaluation_model: 0 seconds


In [10]:
pairs = [ # Test data, List[Tuple[str, str]]
    ("The capital of France is Berlin.", "The capital of France is Paris."),
    ('I am in California', 'I am in United States.'),
    ('I am in United States', 'I am in California.'),
    ("A person on a horse jumps over a broken down airplane.", "A person is outdoors, on a horse."),
    ("A boy is jumping on skateboard in the middle of a red bridge.", "The boy skates down the sidewalk on a red bridge"),
    ("A man with blond-hair, and a brown shirt drinking out of a public water fountain.", "A blond man wearing a brown shirt is reading a book."),
    ("Mark Wahlberg was a fan of Manny.", "Manny was a fan of Mark Wahlberg.")
]

In [11]:
# Prompt the pairs
prompt = "<pad> Determine if the hypothesis is true given the premise?\n\nPremise: {text1}\n\nHypothesis: {text2}"

In [12]:
input_pairs = [prompt.format(text1=pair[0], text2=pair[1]) for pair in pairs]
input_pairs

['<pad> Determine if the hypothesis is true given the premise?\n\nPremise: The capital of France is Berlin.\n\nHypothesis: The capital of France is Paris.',
 '<pad> Determine if the hypothesis is true given the premise?\n\nPremise: I am in California\n\nHypothesis: I am in United States.',
 '<pad> Determine if the hypothesis is true given the premise?\n\nPremise: I am in United States\n\nHypothesis: I am in California.',
 '<pad> Determine if the hypothesis is true given the premise?\n\nPremise: A person on a horse jumps over a broken down airplane.\n\nHypothesis: A person is outdoors, on a horse.',
 '<pad> Determine if the hypothesis is true given the premise?\n\nPremise: A boy is jumping on skateboard in the middle of a red bridge.\n\nHypothesis: The boy skates down the sidewalk on a red bridge',
 '<pad> Determine if the hypothesis is true given the premise?\n\nPremise: A man with blond-hair, and a brown shirt drinking out of a public water fountain.\n\nHypothesis: A blond man wearing

In [13]:
# Use text-classification pipeline to predict
classifier = pipeline(
    "text-classification",
    model=model_id,  # 'vectara/hallucination_evaluation_model',
    tokenizer=AutoTokenizer.from_pretrained('google/flan-t5-base'),
    trust_remote_code=True
)

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
Device set to use cpu


In [14]:
full_scores = classifier(input_pairs, top_k=None)  # List[List[Dict[str, float]]]

In [15]:
# Optional: Extract the scores for the 'consistent' label
simple_scores = [score_dict['score'] for score_for_both_labels in full_scores for score_dict in score_for_both_labels if score_dict['label'] == 'consistent']

In [16]:
print(f"simple_scores: {simple_scores}")
# Expected output: [0.011061512865126133, 0.6473632454872131, 0.1290171593427658, 0.8969419002532959, 0.18462494015693665, 0.005031010136008263, 0.05432349815964699]

simple_scores: [0.011061499826610088, 0.6473637223243713, 0.1290174126625061, 0.8969420790672302, 0.18462441861629486, 0.005031016655266285, 0.05432353541254997]


In [17]:
print(f"full_scores: {full_scores}")

full_scores: [[{'label': 'hallucinated', 'score': 0.988938570022583}, {'label': 'consistent', 'score': 0.011061499826610088}], [{'label': 'consistent', 'score': 0.6473637223243713}, {'label': 'hallucinated', 'score': 0.3526362180709839}], [{'label': 'hallucinated', 'score': 0.8709826469421387}, {'label': 'consistent', 'score': 0.1290174126625061}], [{'label': 'consistent', 'score': 0.8969420790672302}, {'label': 'hallucinated', 'score': 0.1030578762292862}], [{'label': 'hallucinated', 'score': 0.815375566482544}, {'label': 'consistent', 'score': 0.18462441861629486}], [{'label': 'hallucinated', 'score': 0.9949689507484436}, {'label': 'consistent', 'score': 0.005031016655266285}], [{'label': 'hallucinated', 'score': 0.9456764459609985}, {'label': 'consistent', 'score': 0.05432353541254997}]]
